#Loading Libraries

In [ ]:
import numpy as np
import pandas as pd
import pydub
import os
import re
from scipy.io.wavfile import read, write

#Loading the dataset

In [ ]:
#training dataset
train_path  = '../dataset/wavLITE/train/'
errors = []
complete = 0
dataset = []
for filename in os.listdir(train_path):
    try:
        fs, y = read(train_path+'true/'+filename)
        dataset = np.array([y,1])
        complete = complete + 1
    else:
        errors = errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(train_path)),'%')

complete = 0
for filename in os.listdir(train_path):
    try:
        fs, y = read(train_path+'false/'+filename)
        dataset = np.array([y,0])
        complete = complete + 1
    else:
        errors = errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(train_path)),'%')

print("Script complete with ", len(errors), " errors.")
if len(errors):
    for error in errors:
        print(error)

In [ ]:
#test dataset
test_data = []
test_path  = '../dataset/wavLITE/test/'
for filename in os.listdir(test_path):
    try:
        fs, y = read(test_path+'true/'+filename)
        dataset = np.array([y,1])
        complete = complete + 1
    else:
        errors = errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(test_path)),'%')

complete = 0
for filename in os.listdir(test_path):
    try:
        fs, y = read(test_path+'false/'+filename)
        dataset = np.array([y,0])
        complete = complete + 1
    else:
        errors = errors.append(filename)
    print('Complete: ',(complete*100)/len(os.listdir(test_path)),'%')

print("Script complete with ", len(errors), " errors.")
if len(errors):
    for error in errors:
        print(error)

In [ ]:
#Convert dataset list of numpy arrays to list of tensors
for data in dataset:    
    tf.convert_to_tensor(data)

In [ ]:
y.dtype()

In [ ]:
write('original.wav', 44100, y)